In [207]:
batch_size=60

In [208]:
import numpy as np  
# import dill
import matplotlib.pyplot as plt  
import pandas as pd
eps = np.finfo(float).eps
from numpy import log2 as log
import math
import sys
from scipy.linalg import svd

# Read dataset to pandas dataframe
dataset = pd.read_csv('/home/aishwarya/CSIS/SMAI/SMAI_assig/a-5/small_Data.csv')

In [209]:
train, validate = np.split(dataset, [int(.8*len(dataset))]) #for sequential data
# train, validate = np.split(dataset.sample(frac=1), [int(.8*len(dataset))]) # for random 

# dataset1

In [214]:
def prepare_data_X_Y(data1):
    X= data1.iloc[:,:-1].values
    Y= data1.iloc[:,-1].values
    return X,Y
    
x_train,Y_train=prepare_data_X_Y(train)
x_validate,y_validate=prepare_data_X_Y(validate)


len(x_train[0])
# X=dataset1
# X=(X - dataset.mean().values)/dataset.std().values
# type(X)
# X
# x_train

784

In [211]:
import numpy as np
# Lambda functions #

# Sigmoid activation and its derivative
SIG = lambda x : 1 / (1 + np.exp(-x))
dSIG = lambda o : np.multiply(o, (1 - o))

# Error function and its derivative
ERROR = lambda t, o : np.multiply(0.5, np.multiply((t - o), (t - o)))
dERROR = lambda t, o : o - t

class Layer(object):
    def __init__ (self, inNodes, outNodes, alpha):
        # Numer of input and output nodes
        self.inNodes = inNodes + 1
        self.outNodes = outNodes
        self.alpha = alpha

        # Matrix of weights init + bias weight
        self.weights = np.round(np.random.rand(self.inNodes, self.outNodes) - 0.5, 2)
    #         print np.round(np.random.rand(inNodes, outNodes) - 0.5, 2)
    #         print 'self-weights='
    #         print self.weights


    def fwd (self, inputs):
        # Add the bias value to the input
        self.input = np.concatenate((inputs, [1]))#, axis=1)
        print 'self input=',self.input

        # Sum the inputs and normalize for each output
        sums = np.dot(self.input, self.weights)
        self.output = SIG(sums)

        return self.output
    
    def bck (self, dL1):
        # Derivative of L1 /w respect to the sum
        print self.output.shape
        dSIG_OUT = dSIG(self.output)
        print 'f`z-', dSIG_OUT
        print dSIG_OUT.shape
        print 'y-y`-', dL1
        print dL1.shape
        
        dL1_SUM = np.multiply(dL1, dSIG_OUT)
        print '\n(y-y^)*f`(z)==',dL1_SUM
        print dL1_SUM.shape

        # Derivative of L1 /w respect to input (to be passed to L0)
        W_T = np.transpose(self.weights)
        print '\n\nweights=',W_T
        dL1_L0 = np.dot(dL1_SUM, W_T)
        print '\n\ndL1_L0-', dL1_L0
        print 'dL1_L0-', dL1_L0.shape
#         dL1_L0=np.asarray(dL1_L0)
        dL1_L0=dL1_L0.reshape(dL1_L0.shape[0],1)
        print dL1_L0.shape
        dL1_L0 = np.delete(dL1_L0, -1, 0)
        print 'll----------', dL1_L0.shape

        # Change the weights using derivative of L1 /w respect to W
        input_T = np.transpose(self.input)
        
        print 'input ki shape-',input_T.shape
        input_T=input_T.reshape(input_T.shape[0],1)
        dL1_SUM=dL1_SUM.reshape(dL1_SUM.shape[0],1)
        dL1_SUM=dL1_SUM.transpose()
        
        print self.weights.shape
        print 'dL1_SUM-',dL1_SUM.shape
        dL1_W = np.dot(input_T, dL1_SUM)
        print 'self-alpha-', self.alpha
        self.weights -= self.alpha * dL1_W
        return dL1_L0
    

In [212]:
class NeuralNetwork(object):
    def __init__ (self, features, hiddenNeurons, classes, hiddenLayers = 1, alpha = 2):
        # Create the first layer
        self.layerStack = np.array([Layer(features, hiddenNeurons, alpha)])

        # Create the hidden layers
        for x in range(hiddenLayers - 1):
            self.layerStack = np.append(self.layerStack, [Layer(hiddenNeurons, hiddenNeurons, alpha)])

        # Create the output layer
        self.layerStack = np.append(self.layerStack, [Layer(hiddenNeurons, classes, alpha)])

    def eval(self, inputs):
        # Forward the signal through the layers
        lastInput = inputs
        for l in self.layerStack:
            lastInput = l.fwd(lastInput)
        ans=np.argmax(lastInput)
        print 'outs-',lastInput
        print 'outputs-',ans

        return lastInput

#     def train(self):
#         print 'll'
        
        
    def train(self,inputs,target):
        print 'hi its train'
        for i in range(1000):
            j=0
            while j+batch_size<=len(inputs):
                print 'j=',j
                inputVector=inputs[j]
                out=self.eval(inputVector)
                print 'predicted y-hat-',out

                t=target[j]
                y=np.zeros(10, dtype = int)
                y[t]=1
                print 'actual y is-',y
                errorVector=ERROR(y,out)
                print 'error i=',errorVector
                errorDerivative=dERROR(out,y)
                print '\nerror deriv-',errorDerivative
#                 print '\nout-',out
#                 output=np.argmax(out)
#                 print(output)
                print '\n====going to back prop============'
                for l in range(len(self.layerStack)-1, -1 , -1):
                    print '==>> layer[',l,']'            
                    errorDerivative=self.layerStack[l].bck(errorDerivative)
                    print 'errorDerivs recvd',errorDerivative
                j+=batch_size





In [213]:
# XOR Test
i = np.matrix([[0, 0],
	       [1, 0],
	       [0, 1],
	       [1, 1]])
t = np.matrix([[1],
 	       [0],
	       [0],
 	       [1]])
# Run
nn = NeuralNetwork(784, 10, 10)
nn.train(x_train, Y_train)
# while True:
# 	# Get input and return evaluation
# 	sample = input("Enter an array: ")
# 	sample = np.matrix(sample)
# 	print(np.round(nn.eval(sample)))


hi its train
j= 0
self input= [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0  16  51  57  18   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0  50  19   0 165 193 137 165
 176 185 166 156 172 174 172 173 172 186 183 193 186 184 181 176 175 174
 176 195 236 209 206 151 151 212 152 141 105  89 123 114 108 117 129 132
 125 125 120 133 128 123 125 131 132 136 145 144 162 173 224 118  79 192
 200 130 198 216 161 172 163 143 136 136 123 130 134 133 141 147 165 158
 168 180 202 166 183 201 227  67   0  77  92  19  23 199 105 173 242 204
 215 206 202 151 137 

/home/aishwarya/.local/lib/python2.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in exp
  """


ValueError: cannot reshape array of size 7850 into shape (10,1)

In [ ]:
epochs=[0,1,2,3,4,5,6,7,8,9]
plt.plot(epochs,errors)
plt.xlabel("epochs in 10000's")
plt.ylabel('error')
plt.show()